# Clustering

In [ ]:
import pandas as pd
import numpy as np

from hit_prediction_code import common
from hit_prediction_code.dataloaders import matcher

ab_cleaned = pd.read_parquet('../data/hit_song_prediction_ismir2020/processed/msd_bb_mbid_cleaned_matches_ab_unique.parquet')
ess_cleaned = pd.read_parquet('../data/hit_song_prediction_ismir2020/processed/msd_bb_mbid_cleaned_matches_essentia_unique.parquet')
numeric_cols = matcher.get_numeric_columns(ab_cleaned)
features = set(common.get_columns_matching_list(numeric_cols, common.hl_no_year_list()))

ab_cleaned['peakPos'] = ab_cleaned['peakPos'].fillna(150)
ess_cleaned['peakPos'] = ess_cleaned['peakPos'].fillna(150)

In [ ]:
from sklearn import manifold

x = ab_cleaned

dim_red = manifold.TSNE(n_components=2, init='pca', random_state=0)
data = pd.DataFrame(dim_red.fit_transform(x[features]))

In [ ]:
data['peakPos'] = x['peakPos']
data['hit'] = (data['peakPos'] <= 100).apply(lambda v: 'red' if v else 'gray')
display(data.plot.scatter(x=0, y=1, c='peakPos', colormap='Spectral', title='TSNE peak pos', figsize=(15,10)))
display(data.plot.scatter(x=0, y=1, c='hit', colormap='Spectral', title='TSNE hits', figsize=(15,10)))